In [1]:
# import flask
from flask import Flask, jsonify, request

from src.models import Station, StationName, StationLocation
from geopy import distance

In [2]:
# create flask app
app = Flask(__name__)

# endpoint for gathering a list of stations
@app.route('/stations')
def stations():
    stations = (Station.select(
                    Station, 
                    StationName.name,
                    StationLocation.latitude, 
                    StationLocation.longitude,
                    StationLocation.altitude)
                .join(StationName, on=(
                        (StationName.station_id == Station.id)
                        &StationName.to_date.is_null()    
                        ))
                .join(StationLocation, on=(
                        (StationLocation.station_id == Station.id)
                        &StationLocation.to_date.is_null()    
                        ))
                .dicts())
    return jsonify(list(stations))

@app.route('/stations/<int:station_id>')
def station(station_id):
    station = (Station.select(
                    Station, 
                    StationName.name, 
                    StationLocation.latitude, 
                    StationLocation.longitude, 
                    StationLocation.altitude)
                .join(StationName, on=(
                        (StationName.station_id == Station.id)
                        &StationName.to_date.is_null()    
                        ))
                .join(StationLocation, on=(
                        (StationLocation.station_id == Station.id)
                        &StationLocation.to_date.is_null()    
                        ))
                .where(Station.id == station_id)
                .dicts())
    return jsonify(list(station))

@app.route('/stations/radius')
def stations_radius():
    latitude = request.args.get('latitude', default=52.520008, type=float)
    longitude = request.args.get('longitude', default=13.404954, type=float)
    length = request.args.get('length', default=25, type=float)
    stations = Station.get_as_dicts()
    
    # filter it by distance to given latitude and longitude
    stations = filter(lambda station: distance.distance(
                        (station['latitude'], station['longitude']), 
                        (latitude, longitude)).kilometers <= length, 
                        stations)
    return jsonify(list(stations))

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Sep/2023 23:27:21] "GET /stations HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2023 23:28:01] "GET /stations/3811 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2023 23:28:53] "GET /stations/radius?latitude=52.520008&lingitude=13.404954&length=50 HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2023 23:29:02] "GET /stations/radius?latitude=52.520008&lingitude=13.404954&length=10 HTTP/1.1" 200 -
